# Wrangling of the Card Data

Here I will download and clean the data for MTG cards.

First we will download the data from [MTGJSON](https://mtgjson.com/downloads/all-files/).  The `AllPrintings` card data comes in various formats, such as json, sql, csv, and parquet.

I will use the [parquet format](https://parquet.apache.org/), since that is the most performant format for data analysis.  It has high compression, fast load times, and can query directly on disk.  This minimizes both disk space and memory usage.
https://mtgjson.com/api/v5/AllPrintingsParquetFiles.tar.gz


In [4]:
import os
from datetime import datetime

save_path = "~/mtg-modeling/data/raw/mtgjson/"
save_path = os.path.expanduser(save_path)  # Expand the user path (~) to full path
tar_file = "AllPrintingsParquetFiles.tar.gz"
url = f"https://mtgjson.com/api/v5/{tar_file}"
tar_path = os.path.join(save_path, tar_file)

os.makedirs(save_path, exist_ok=True)  # Create the directory if it doesn't exist

# Download the file
os.system(f"wget -P {save_path} --progress=dot:giga {url}")

# Extract the file
os.system(f"tar -xzf {tar_path} -C {save_path}")

# Remove the original tar.gz file
os.system(f"rm {tar_path}")

# Print the current date and time
print(f"Current datetime: {datetime.now()}")

# Disk usage
print("Disk usage:")
os.system(f"du -sh {save_path}")

--2024-08-14 07:52:15--  https://mtgjson.com/api/v5/AllPrintingsParquetFiles.tar.gz
Resolving mtgjson.com (mtgjson.com)... 104.21.64.186, 172.67.154.80
Connecting to mtgjson.com (mtgjson.com)|104.21.64.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120351414 (115M) [application/octet-stream]
Saving to: ‘/root/mtg-modeling/data/raw/mtgjson/AllPrintingsParquetFiles.tar.gz’

     0K ........ ........ ........ ........ 27% 3.22M 26s
 32768K ........ ........ ........ ........ 55% 3.15M 16s
 65536K ........ ........ ........ ........ 83% 3.16M 6s
 98304K ........ ........ ..               100% 3.10M=36s

2024-08-14 07:52:51 (3.16 MB/s) - ‘/root/mtg-modeling/data/raw/mtgjson/AllPrintingsParquetFiles.tar.gz’ saved [120351414/120351414]



Current datetime: 2024-08-14 07:52:52.776967
Disk usage:
143M	/root/mtg-modeling/data/raw/mtgjson/


0

We have 18 parquet files associated with the card data, let't take a quick tour.  I will look at each file in order.

In [5]:
import os
import pathlib

path = pathlib.Path('../../data/raw/mtgjson/AllPrintingsParquetFiles')

# list of files
files = os.listdir(path)
files.sort()
files

['cardForeignData.parquet',
 'cardIdentifiers.parquet',
 'cardLegalities.parquet',
 'cardPrices.parquet',
 'cardPurchaseUrls.parquet',
 'cardRulings.parquet',
 'cards.parquet',
 'meta.parquet',
 'setBoosterContentWeights.parquet',
 'setBoosterContents.parquet',
 'setBoosterSheetCards.parquet',
 'setBoosterSheets.parquet',
 'setTranslations.parquet',
 'sets.parquet',
 'tokenIdentifiers.parquet',
 'tokens.parquet']

# Unique Identifiers

Most of the files have a `uuid`. This is the universally unique identifier (UUID v5) for each card printing.  It is the primary key for the `cards.parquet` file and will be used to join data across tables.


Properties of the `uuid`:
 - Reprinted card editions: Unique id
 - [Double-faced cards](https://mtg.fandom.com/wiki/Double-faced_card) (DBC): Each face has a unique `uuid`.
 - Foreign languages: Same Id. 

Other systems with identifiers to be aware of:
 - `multiverseId`: The WOTC card identifier used their [Gatherer](https://gatherer.wizards.com) card database.  
    - Reprinted card editions: Unique id
    - Double-faced cards: Same id
    - Foreign languages: Different id
 - `scryfallId`: The [Scryfall](https://scryfall.com/) uuid.  It has different rules than the MTGJSON uuid, such as faces of DFCs are not unique.
    - Reprinted card editions: Unique id
    - Double-faced cards: Same id.  See `scryfallCardBackId`.
    - Foreign languages: Different id

# Explore Tables
## Foreign Data

The `cardForeignData.parquet` file contains the [Foreign Data](https://mtgjson.com/data-models/foreign-data/).  It has the translated card content for alternate languages beyond english.

There are a few non-obvious field names.  
 - `faceName`: For [double-faced cards](https://mtg.fandom.com/wiki/Double-faced_card) (DBC), the name of a given face.

The data is loaded below.

In [51]:
import pandas as pd

def load_by_filename(file):
    print(f"Opening {file}")
    filepath = path / file
    df = pd.read_parquet(filepath)
    print(f"Loaded DF of shape: {df.shape}")
    return df

file = files[0]
df = load_by_filename(file)
df.head()

Opening cardForeignData.parquet
Loaded DF of shape: (370003, 9)


,faceName,flavorText,identifiers,language,multiverseId,name,text,type,uuid
0,None,"„Es ist der Wille aller, und meine Hand, die i...","{""multiverseId"": 148411, ""scryfallId"": ""c8d214...",German,148411.0,Ausgewählter der Ahnfrau,Erstschlag (Diese Kreatur fügt Kampfschaden vo...,"Kreatur — Mensch, Kleriker",5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c
1,None,"""La voluntad de todos, realizada por mi mano.""","{""multiverseId"": 150317, ""scryfallId"": ""3f227a...",Spanish,150317.0,Elegido de la Antepasada,Daña primero. (Esta criatura hace daño de comb...,Criatura — Clérigo humano,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c
2,None,« La volonté de tous passe par ma main. »,"{""multiverseId"": 149934, ""scryfallId"": ""03da12...",French,149934.0,Élu de l'Ancêtre,Initiative (Cette créature inflige des blessur...,Créature : humain et clerc,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c
3,None,"""La volontà di tutti, eseguita per mano mia.""","{""multiverseId"": 148794, ""scryfallId"": ""3d078c...",Italian,148794.0,Prescelto dell'Antenata,Attacco improvviso (Questa creatura infligge d...,Creatura — Chierico Umano,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c
4,None,すべての意思を、この手で成そう。,"{""multiverseId"": 148028, ""scryfallId"": ""6eac08...",Japanese,148028.0,祖神に選ばれし者,先制攻撃 （このクリーチャーは先制攻撃を持たないクリーチャーよりも先に戦闘ダメージを与える。...,クリーチャー — 人間・クレリック,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c


Here we can see that each language of the `Boom // Bust` card has unique `multiverseId` and common `uuid`

In [52]:
mask = df['name'] == 'Boom // Bust'
mask &= df['uuid'] == '7104f01e-0ca6-53e6-873f-ee246015174a'
df.loc[mask, ['faceName', 'name', 'language', 'multiverseId', 'identifiers', 'uuid']].head()

,faceName,name,language,multiverseId,identifiers,uuid
285143,Boom,Boom // Bust,German,140480.0,"{""multiverseId"": 140480, ""scryfallId"": ""303895...",7104f01e-0ca6-53e6-873f-ee246015174a
285144,Boom,Boom // Bust,Spanish,141200.0,"{""multiverseId"": 141200, ""scryfallId"": ""d535b1...",7104f01e-0ca6-53e6-873f-ee246015174a
285145,Boom,Boom // Bust,French,140360.0,"{""multiverseId"": 140360, ""scryfallId"": ""ee6813...",7104f01e-0ca6-53e6-873f-ee246015174a
285146,Boom,Boom // Bust,Italian,140600.0,"{""multiverseId"": 140600, ""scryfallId"": ""5f8db4...",7104f01e-0ca6-53e6-873f-ee246015174a
285147,Boom,Boom // Bust,Japanese,140840.0,"{""multiverseId"": 140840, ""scryfallId"": ""4af0dd...",7104f01e-0ca6-53e6-873f-ee246015174a


Here we can see that each printing of the `Boom // Bust` card has a common `multiverseId` and unique `uuid` across the Boom and Bust faces.

In [53]:
mask = df['name'] == 'Boom // Bust'
mask &= df['multiverseId'] == 140480.0
df.loc[mask, ['faceName', 'name', 'language', 'multiverseId', 'identifiers', 'uuid']].head()

,faceName,name,language,multiverseId,identifiers,uuid
285143,Boom,Boom // Bust,German,140480.0,"{""multiverseId"": 140480, ""scryfallId"": ""303895...",7104f01e-0ca6-53e6-873f-ee246015174a
285151,Bust,Boom // Bust,German,140480.0,"{""multiverseId"": 140480, ""scryfallId"": ""303895...",94a1683f-deed-5731-bc0a-894075b3fdb2


Most cards are printed in 10 languages beyond english.

In [33]:
df['uuid'].value_counts().value_counts()

count
10    13194
6      9757
7      8048
8      5111
5      4187
4      3679
9      3237
3      3082
1      3009
2      2629
Name: count, dtype: int64

It appears that Phyrexian is not compatible with the UTF-8 character encoding!

In [12]:
df.loc[df['language'] == 'Phyrexian', 'text'].head()

116703    |zvEZYCsDFhg.\n|besYf,GvEng,mpreLYv co,LFr2aho...
116704    |pcEnusL.\n+Ⅰ: |co,tuhtZFsDYt,DEEtAGvEnyg,hruP...
157935    |zvEpnrYhm.\n|su,Lecc,CF,CsYPYcs,usFPuuDs²\\²M...
162944                                                 None
169505    |zvEhiUIpenDk zvE&EhclYvh.\n|beCEcs,FgCnUCA Fg...
Name: text, dtype: object

## Identifiers

The `cardIdentifiers.parquet` file contains the [Identifiers](https://mtgjson.com/data-models/identifiers/).  These are used to map the MTGJSON card to other platforms, such as TCG Player or MTG Arena.

The other platforms are:  
 - [Card Kingdom](www.cardkingdom.com): A large-scale card vender
 - [Card Sphere](https://www.cardsphere.com/): A card trading marketplace
 - [Cardmarket](https://www.cardmarket.com/en/Magic): A card marketplace
 - [MTG Arena](https://magic.wizards.com/en/mtgarena): An online environment to play MTG
 - [MTG Online](https://magic.wizards.com/en/mtgo): An online environment to play MTG
 - [WOTC Gather](https://gatherer.wizards.com/): WOTC's official data source.
 - [Scryfall](https://scryfall.com/): Another MTG data source.  Note DFC do not have unique UUIDs.
 - [TGC Player](https://www.tcgplayer.com/): A card marketplace

A note on the `uuids`.  The foil and non-foil versions of a card have unique `uuids`.  To cross-reference the other foil type, use the `mtgjsonFoilVersionId` and `mtgjsonNonFoilVersionId` fields.

In [47]:
file = files[1]
df = load_by_filename(file)
df.loc[2]

Opening cardIdentifiers.parquet
Loaded DF of shape: (96203, 21)


cardKingdomEtchedId                                         None
cardKingdomFoilId                                           None
cardKingdomId                                             122720
cardsphereFoilId                                              21
cardsphereId                                                  22
mcmId                                                      16166
mcmMetaId                                                    176
mtgArenaId                                                  None
mtgjsonFoilVersionId        8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a
mtgjsonNonFoilVersionId                                     None
mtgjsonV4Id                 9eb2e54c-a12b-5e88-a9c0-d8c84c52d59c
mtgoFoilId                                                 26993
mtgoId                                                     26992
multiverseId                                              129465
scryfallCardBackId          0aeebaf5-8c7d-4636-9e82-8c27447861f7
scryfallId               

In [50]:
uuids = ['57aaebc1-850c-503d-9f6e-bb8d00d8bf7c', '8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a']
mask = df['uuid'].isin(uuids)
df.loc[mask, ['mtgjsonFoilVersionId', 'mtgjsonNonFoilVersionId', 'uuid']]

,mtgjsonFoilVersionId,mtgjsonNonFoilVersionId,uuid
2,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,None,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c
3,None,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a


In [6]:
# Define the SQL command to create the view
drop_view = """
DROP VIEW IF EXISTS standard_cards_view;
"""

create_view_sql = """
CREATE VIEW standard_cards_view AS
SELECT
    cl.standard, 
    c.name, 
    c.setCode, 
    cs.releaseDate,
    c.number, 
    c.layout, 
    c.*
FROM cards AS c
JOIN cardLegalities AS cl ON cl.uuid = c.uuid
JOIN cardPurchaseUrls AS cp ON cp.uuid = c.uuid
JOIN sets AS cs ON cs.code = c.setCode
WHERE cl.standard = 'Legal' 
    AND c.setCode = 'OTJ'
    AND c.isPromo IS NULL 
    AND c.borderColor = 'black' 
    AND c.isReprint IS NULL 
    AND c.promoTypes IS NULL 
ORDER BY c.name ASC, cs.releaseDate ASC, c.power DESC;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(drop_view))
    connection.execute(text(create_view_sql))

print("View 'standard_cards_view' created successfully.")

View 'standard_cards_view' created successfully.


In [12]:
# Define the SQL command to create the view
drop_view = """
DROP VIEW IF EXISTS OTJ_set_view;
"""

create_view_sql = """
CREATE VIEW OTJ_set_view AS
SELECT
    cl.standard, 
    c.name, 
    c.setCode, 
    cs.releaseDate,
    c.number, 
    c.layout, 
    c.*
FROM cards AS c
JOIN cardLegalities AS cl ON cl.uuid = c.uuid
JOIN cardPurchaseUrls AS cp ON cp.uuid = c.uuid
JOIN sets AS cs ON cs.code = c.setCode
WHERE c.setCode IN ('OTJ', 'BIG', 'OTP')
ORDER BY c.name ASC, cs.releaseDate ASC, c.power DESC;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(drop_view))
    connection.execute(text(create_view_sql))

print("View 'OTJ_set_view' created successfully.")

View 'OTJ_set_view' created successfully.


In [16]:

# Define the SQL query
sql_query = """
WITH ranked_cards AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY name ORDER BY releaseDate ASC, power DESC) AS rn
    FROM OTJ_set_view
)
SELECT
    name, 
    setCode, 
    releaseDate,
    number, 
    layout,
    availability,
    power, toughness,
    colorIdentity, colors,
    types, subtypes, supertypes,
    manaCost, manaValue,
    edhrecRank, edhrecSaltiness
    -- text, flavorText
FROM ranked_cards
WHERE rn = 1
ORDER BY name ASC;
"""

# Execute the query and load the results into a DataFrame
with engine.connect() as connection:
    df = pd.read_sql_query(sql_query, connection)

# Display the DataFrame
df.to_feather('../../data/OTJ/card.feather')
print(df.shape)
df.head()

(371, 17)


,name,setCode,releaseDate,number,layout,availability,power,toughness,colorIdentity,colors,types,subtypes,supertypes,manaCost,manaValue,edhrecRank,edhrecSaltiness
0,Abraded Bluffs,OTJ,2024-04-19,251,normal,"arena, mtgo, paper",None,None,"R, W",,Land,Desert,,None,0.0,5898.0,NaN
1,Abrupt Decay,OTP,2024-04-19,34,normal,"arena, mtgo, paper",None,None,"B, G","B, G",Instant,,,{B}{G},2.0,625.0,0.31
2,Akul the Unrepentant,OTJ,2024-04-19,346,normal,"arena, mtgo, paper",5,5,"B, R","B, R",Creature,"Scorpion, Dragon, Rogue",Legendary,{B}{B}{R}{R},4.0,12398.0,NaN
3,Aloe Alchemist,OTJ,2024-04-19,152,normal,"arena, mtgo, paper",3,2,G,G,Creature,"Plant, Warlock",,{1}{G},2.0,17373.0,NaN
4,Ambush Gigapede,OTJ,2024-04-19,77,normal,"arena, mtgo, paper",6,2,B,B,Creature,Insect,,{4}{B}{B},6.0,21172.0,NaN
